In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import ipywidgets as widgets
from IPython.display import display

In [2]:
import calculator_functions as cf
import helper_functions as hf

In [3]:
def select_rows(df, cols = 5):
    checkboxes = []
    for i, row in df.iterrows():
        checkbox = widgets.Checkbox(
            value = False,
            description = f'{row["Food"]}',
        )
        checkboxes.append(checkbox)

    grid = widgets.GridBox(checkboxes, layout = widgets.Layout(
        grid_template_columns = f'repeat({cols}, 150px)',
        grid_gap='10px 10px'
    ))

    def on_button_click(b):
        selected_rows = [i for i, checkbox in enumerate(checkboxes) if checkbox.value]
        selected_df = df.iloc[selected_rows]
        display(selected_df)
        global data
        data = selected_df

    button = widgets.Button(description = "Submit")
    button.on_click(on_button_click)

    display(grid, button)

In [4]:
gender = input("Enter your gender (m/f): ")
weight = hf.input_validation("Enter your weight (in kg): ", int)
height = hf.input_validation("Enter your height (in cm): ", int)
age = hf.input_validation("Enter your age (in years): ", int)
activity_multiplier = hf.input_validation("Enter your activity multiplier: ", float)
goal = input("Enter your goal (cutting/bulking/maintaining): ")

In [5]:
bmr = cf.bmr(gender, weight, height, age)
tdee = cf.tdee(bmr, activity_multiplier)
daily_caloric_intake = {
    "cutting": 0.75 * tdee,
    "bulking": 1.10 * tdee,
    "maintaining": tdee,
}.get(goal, tdee)

protein, carbohydrates, fats, fibre, saturated_fats = cf.macros(
    int(daily_caloric_intake), goal
)

In [ ]:
print(f"BMR: {bmr}")
print(f"TDEE: {tdee}")
print(f"Daily Caloric Intake: {int(daily_caloric_intake)}")
print(f"Protein (g): {protein}")
print(f"Carbohydrates (g): {carbohydrates}")
print(f"Fats (g): {fats}")
print(f"Fibre (g): {fibre}")
print(f"Saturated Fats (g): {saturated_fats}")

In [7]:
nutrients_data = hf.create_nutrients_df()
nutrient_bounds = cf.nutrient_bounds(age, gender)

In [ ]:
food_items = pd.read_csv("food_items.csv")
select_rows(food_items, cols = 5)

In [9]:
c = data["Price"].values

A_ub = []
b_ub = []

A_ub.extend(
    [
        -data["Protein (g)"].values,
        data["Protein (g)"].values,
        -data["Carbohydrates (g)"].values,
        data["Carbohydrates (g)"].values,
        -data["Fats (g)"].values,
        data["Fats (g)"].values,
        -data["Fibre (g)"].values,
        data["Fibre (g)"].values,
        data["Saturated Fats (g)"].values,
    ]
)

b_ub.extend(
    [
        -protein,
        protein * 1.01,
        -carbohydrates,
        carbohydrates * 1.01,
        -fats,
        fats * 1.01,
        -fibre,
        fibre * 1.01,
        saturated_fats,
    ]
)

for nutrient in nutrients_data.columns[1:]:
    if nutrient.endswith("_RDA"):
        base_nutrient = nutrient[:-4]
        if base_nutrient in data.columns:
            A_ub.append(-data[base_nutrient].values)
            b_ub.append(-nutrient_bounds[nutrient])
    elif nutrient.endswith("_UL"):
        base_nutrient = nutrient[:-3]
        if base_nutrient in data.columns:
            A_ub.append(data[base_nutrient].values)
            b_ub.append(nutrient_bounds[nutrient])

A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

bounds = [(0, None) for _ in range(len(data))]

In [ ]:
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

if result.success:
    print("Optimization succeeded!")
else:
    print("Optimization failed! No feasible solution found.")

In [ ]:
if result.success:
    servings = np.round(result.x, 1)
    quantity = np.round(servings * data["Serving (g)"], 1)
    total_cost_per_item = np.round(servings * data["Price"], 1)

    nutrient_totals = {}
    for nutrient in data.columns[3:]:
        nutrient_totals[nutrient] = np.round(np.sum(servings * data[nutrient]), 1)

    result_df = pd.DataFrame(
        {
            "Food Item": data["Food"],
            "Optimal Servings": servings,
            "Optimal Quantity": quantity,
            "Total Cost": total_cost_per_item,
            **{
                f"Total {nutrient}": np.round(servings * data[nutrient], 1)
                for nutrient in data.columns[3:]
            },
        }
    )
    result_df["Optimal Quantity"] = result_df["Optimal Quantity"].apply(
        hf.format_weight
    )
    print(result_df[result_df.iloc[:, 1:].ne(0).any(axis=1)])

In [ ]:
if result.success:
    print("\nDaily Summary:")
    for nutrient, total in nutrient_totals.items():
        print(f"{nutrient}: {total}")

In [ ]:
if result.success:
    print(f"\nTotal Cost: ${np.sum(total_cost_per_item):.2f}")